In [3]:
import os
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.metrics import (
    accuracy_score, log_loss, roc_auc_score,
    roc_curve, brier_score_loss
)
from sklearn.calibration import calibration_curve

# Import custom models
# Removed old path hack; using package imports))
from llm_prior_project.priors.target_informed_model import TargetInformedModel
from llm_prior_project.priors.target_elicitor import LLMTargetElicitor

np.random.seed(42)


In [34]:
# loading the data 
def load_heart_dataset(path, features, outcome="num"):
    columns = [
        "age", "sex", "cp", "trestbps", "chol", "fbs", "restecg",
        "thalach", "exang", "oldpeak", "slope", "ca", "thal", "num"
    ]
    df = pd.read_csv(path, header=None, names=columns, na_values="?")
    df[outcome] = (df[outcome] > 0).astype(int)
    df = df[features + [outcome]].dropna()
    return df[features], df[outcome]

features = ["age", "sex", "trestbps", "chol", "thalach", "oldpeak", "cp", "exang", "fbs", "restecg"]

X, y = load_heart_dataset("data/heart+disease/processed.hungarian.data", features)
X_cleveland, y_cleveland = load_heart_dataset("data/heart+disease/processed.cleveland.data", features)



X = pd.get_dummies(X, columns=["cp", "restecg"], drop_first=True).astype(float)
X_cleveland = pd.get_dummies(X_cleveland, columns=["cp", "restecg"], drop_first=True).astype(float)

features = list(X.columns)

print("Hungarian:", X.shape, "Cleveland:", X_cleveland.shape)


Hungarian: (261, 13) Cleveland: (303, 13)


In [22]:
print(X.head())

   age  sex  trestbps   chol  thalach  oldpeak  exang  fbs   cp_2   cp_3  \
0   28    1     130.0  132.0    185.0      0.0    0.0  0.0   True  False   
1   29    1     120.0  243.0    160.0      0.0    0.0  0.0   True  False   
3   30    0     170.0  237.0    170.0      0.0    0.0  0.0  False  False   
4   31    0     100.0  219.0    150.0      0.0    0.0  0.0   True  False   
5   32    0     105.0  198.0    165.0      0.0    0.0  0.0   True  False   

    cp_4  restecg_1.0  restecg_2.0  
0  False        False         True  
1  False        False        False  
3  False         True        False  
4  False         True        False  
5  False        False        False  


In [25]:

# --- Utilities: model evaluation & CV (no plotting) ---

def evaluate_models(model, baseline, X, y):
    """Return core metrics for baseline vs informed model."""
    X = np.asarray(X)
    y = np.asarray(y)

    baseline_probs = baseline.predict_proba(X)[:, 1]
    informed_probs = model.predict(X)

    return {
        "baseline_accuracy": accuracy_score(y, (baseline_probs > 0.5).astype(int)),
        "baseline_log_loss": log_loss(y, baseline_probs),
        "baseline_auc": roc_auc_score(y, baseline_probs),
        "informed_accuracy": accuracy_score(y, (informed_probs > 0.5).astype(int)),
        "informed_log_loss": log_loss(y, informed_probs),
        "informed_auc": roc_auc_score(y, informed_probs),
    }


def cross_val_evaluate(X, y, feature_names, targets, alpha=1.0, n_splits=5):
    """Compare baseline vs target-informed logistic regression using cross-validation."""
    skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)
    rows = []

    for train_idx, test_idx in skf.split(X, y):
        X_train, X_test = X[train_idx], X[test_idx]
        y_train, y_test = y[train_idx], y[test_idx]

        baseline = LogisticRegression(max_iter=1000).fit(X_train, y_train)
        informed = TargetInformedModel(alpha=alpha, model_type="logistic").fit(
            X_train, y_train, feature_names=feature_names, targets=targets
        )

        rows.append(evaluate_models(informed, baseline, X_test, y_test))

    return pd.DataFrame(rows)


def cross_val_grid_alphas(X, y, feature_names, targets, alphas, n_splits=5):
    """Run CV across multiple alpha values for TargetInformedModel; return results, summary, and best alpha."""
    frames = []
    for a in alphas:
        df = cross_val_evaluate(X, y, feature_names, targets, alpha=a, n_splits=n_splits)
        df["alpha"] = a
        frames.append(df)

    results = pd.concat(frames, ignore_index=True)
    summary = results.groupby("alpha").mean(numeric_only=True)
    best_alpha = summary["informed_log_loss"].idxmin()

    print("\nCV summary (means):\n", summary)
    print("\nSelected alpha (by informed log_loss):", best_alpha)

    return results, summary, best_alpha


## Prompt used 


You are an expert in **biostatistics and cardiovascular epidemiology**.  
For the **target-informed logistic regression model** provided below, which predicts coronary artery disease (CAD), your task is to propose and justify suitable target values for the regression coefficients.  

Unlike standard ridge regression, which shrinks coefficients toward **zero**, this method shrinks coefficients toward **pre-specified targets** $\mu = (\mu_1, \ldots, \mu_p)$, based on domain knowledge.  

---

**Model Details:**  
- Response: $y \in \{0,1\}$ (0 = healthy, 1 = CAD)  
- Linear Predictor:  

\[
\eta = \beta_0 
+ \beta_1 \cdot \text{age} 
+ \beta_2 \cdot \text{sex} 
+ \beta_3 \cdot \text{trestbps} 
+ \beta_4 \cdot \text{chol} 
+ \beta_5 \cdot \text{thalach} 
+ \beta_6 \cdot \text{oldpeak} 
+ \beta_7 \cdot \text{fbs} 
+ \beta_8 \cdot \text{exang} 
+ \beta_9 \cdot \text{cp\_2} 
+ \beta_{10} \cdot \text{cp\_3} 
+ \beta_{11} \cdot \text{cp\_4} 
+ \beta_{12} \cdot \text{restecg\_1.0} 
+ \beta_{13} \cdot \text{restecg\_2.0}
\]

- Prediction:  

\[
P(y=1|X) = \frac{1}{1 + \exp(-\eta)}
\]

- Objective Function (Target-Informed Ridge Penalty):  

\[
\min_\beta \; - \log L(\beta \,|\, X,y) + \alpha \sum_{j=1}^p (\beta_j - \mu_j)^2
\]

where $\mu_j$ are the **target values** you will propose.  

---

**Predictor Details:**  
- `age`: in years. CAD risk increases with age.  
- `sex`: categorical (1 = male; 0 = female). Males typically have higher CAD risk.  
- `trestbps`: resting blood pressure (mm Hg on admission). Higher values are a risk factor.  
- `chol`: serum cholesterol level (mg/dl). High cholesterol is a modest risk factor.  
- `thalach`: maximum heart rate achieved during exercise (bpm). Higher capacity is protective.  
- `oldpeak`: ST depression induced by exercise relative to rest (mm). Larger values indicate ischemia and higher CAD risk.  
- `fbs`: fasting blood sugar >120 mg/dl (1 = yes, 0 = no). Proxy for diabetes; diabetes strongly increases CAD risk.  
- `exang`: exercise-induced angina (1 = yes, 0 = no). Presence indicates higher CAD risk.  
- `cp_2`: dummy variable for atypical angina (baseline = typical angina).  
- `cp_3`: dummy variable for non-anginal pain (baseline = typical angina).  
- `cp_4`: dummy variable for asymptomatic chest pain (baseline = typical angina). Asymptomatic patients often have more advanced disease and higher CAD risk.  
- `restecg_1.0`: dummy variable for ST-T wave abnormality on resting ECG (baseline = normal ECG). Abnormalities increase CAD risk.  
- `restecg_2.0`: dummy variable for left ventricular hypertrophy on ECG (baseline = normal ECG). Associated with higher CAD risk.  

---

**Your Response Should:**  

1. **Leverage Knowledge & Simulate Literature Use**  
   - Briefly state how you use your epidemiological knowledge (e.g., Framingham Study, MONICA project, meta-analyses) to inform coefficient expectations.  
   - Do *not* rely on the Cleveland Heart Disease dataset, but instead general domain knowledge.  

2. **Propose Target Coefficients ($\mu_j$)**  
   - Provide target values $\mu_j$ for each predictor, representing the expected effect on the log-odds of CAD.  
   - Justify the direction (positive/negative) and plausible magnitude of each coefficient in natural language.  

3. **Rationale for Each Target**  
   - For each predictor, explain why the coefficient should be positive/negative, and roughly how large, based on prior evidence.  
   - For $\beta_0$, clarify its interpretation (baseline log-odds when all predictors are zero) and how you approximate it.  

4. **Uncertainty & Strength of Belief**  
   - Discuss how confident you are about each target. Which ones are well-established (e.g., age, sex, diabetes, exercise-induced angina), and which are more uncertain (e.g., cholesterol in small hospital samples)?  
   - If useful, you may also describe an alternative “weaker” set of targets closer to zero, to represent more cautious shrinkage.  

---

**Output Format:**  
After providing your reasoning in detail, end your answer with a JSON object summarizing the chosen target values only:  

```json
{
  "targets": {
    "age": ... ,
    "sex": ... ,
    "trestbps": ... ,
    "chol": ... ,
    "thalach": ... ,
    "oldpeak": ... ,
    "fbs": ... ,
    "exang": ... ,
    "cp_2": ... ,
    "cp_3": ... ,
    "cp_4": ... ,
    "restecg_1.0": ... ,
    "restecg_2.0": ...
  }
}

In [31]:
# Taken from the llm chatgpt 5
targets = [0.03, 0.7, 0.01, 0.002, -0.01, 0.45, 0.7, 0.8, -0.5, -0.9, 0.4, 0.25, 0.4]

## OOD testing

In [52]:
# %%
import numpy as np
import pandas as pd
from sklearn.metrics import log_loss, roc_auc_score, accuracy_score

def ood_alpha_search(X, y, features, targets, subgroup_mask, alphas, min_size=20):
    """
    Train on complement (~subgroup), test on subgroup.
    Search alphas for TargetInformedModel and pick best one by log_loss.
    """
    # Split into subgroup (test) and complement (train)
    X_train, y_train = X[~subgroup_mask], y[~subgroup_mask]
    X_test, y_test = X[subgroup_mask], y[subgroup_mask]

    # Sanity check
    if len(y_test) < min_size:
        raise ValueError(f"Subgroup too small: {len(y_test)} samples")
    if len(np.unique(y_test)) < 2:
        raise ValueError("Subgroup has only one class, cannot evaluate")

    results = []

    for alpha in alphas:
        model = TargetInformedModel(alpha=alpha, model_type="logistic").fit(
            X_train.values, y_train.values, feature_names=features, targets=targets
        )
        probs = model.predict(X_test.values)

        metrics = {
            "alpha": alpha,
            "log_loss": log_loss(y_test, probs),
            "auc": roc_auc_score(y_test, probs),
            "accuracy": accuracy_score(y_test, (probs > 0.5).astype(int)),
            "n_test": len(y_test)
        }
        results.append(metrics)

    df = pd.DataFrame(results)
    best_alpha = df.loc[df["log_loss"].idxmin(), "alpha"]

    return df, best_alpha


# Define candidate subgroups
subgroups = {
    "age>=60": X["age"] >= 60,
    "age<45": X["age"] < 45,
    "male": X["sex"] == 1,
    "female": X["sex"] == 0,
    "high_BP": X["trestbps"] >= 150,
    "high_chol": X["chol"] >= 240,
    "fbs=1": X["fbs"] == 1,
    "exang=1": X["exang"] == 1,
    "cp_2": X.get("cp_2", pd.Series(False, index=X.index)) == 1,
    "cp_3": X.get("cp_3", pd.Series(False, index=X.index)) == 1,
    "cp_4": X.get("cp_4", pd.Series(False, index=X.index)) == 1,
    "low_thalach": X["thalach"] <= X["thalach"].quantile(0.25),
    "high_thalach": X["thalach"] >= X["thalach"].quantile(0.75),
    "oldpeak>=2": X["oldpeak"] >= 2.0,
    "restecg_1.0": X.get("restecg_1.0", pd.Series(False, index=X.index)) == 1,
    "restecg_2.0": X.get("restecg_2.0", pd.Series(False, index=X.index)) == 1,
}

alphas = [0.01, 0.1, 0.5, 1, 2, 5, 10, 12, 15, 20, 30]

# Run OOD alpha search per subgroup
results = {}
for name, mask in subgroups.items():
    try:
        df, best_alpha = ood_alpha_search(X, y, features, targets, mask, alphas)
        results[name] = {
            "df": df,
            "best_alpha": best_alpha,
        }
        print(f"✅ Subgroup {name}: best_alpha={best_alpha}")
    except ValueError as e:
        print(f"⚠️ Skipped subgroup {name}: {e}")

# Example: access results
# results["age>=60"]["df"]  -> per-alpha metrics
# results["age>=60"]["best_alpha"] -> chosen alpha

    


⚠️ Skipped subgroup age>=60: Subgroup too small: 10 samples
✅ Subgroup age<45: best_alpha=5.0
✅ Subgroup male: best_alpha=10.0
✅ Subgroup female: best_alpha=20.0
✅ Subgroup high_BP: best_alpha=0.5
✅ Subgroup high_chol: best_alpha=5.0
⚠️ Skipped subgroup fbs=1: Subgroup too small: 19 samples
✅ Subgroup exang=1: best_alpha=5.0
✅ Subgroup cp_2: best_alpha=0.5
✅ Subgroup cp_3: best_alpha=30.0
✅ Subgroup cp_4: best_alpha=12.0
✅ Subgroup low_thalach: best_alpha=10.0
✅ Subgroup high_thalach: best_alpha=2.0
✅ Subgroup oldpeak>=2: best_alpha=1.0
✅ Subgroup restecg_1.0: best_alpha=30.0
⚠️ Skipped subgroup restecg_2.0: Subgroup too small: 6 samples


In [53]:
# Collect valid best alphas and their subgroup sizes
alpha_list = []
weights = []

for name, res in results.items():
    if "best_alpha" in res:  # skip invalid ones
        best_alpha = res["best_alpha"]
        n_test = res["df"]["n_test"].iloc[0]  # subgroup size stored in df
        alpha_list.append(best_alpha)
        weights.append(n_test)

alpha_series = pd.Series(alpha_list, index=[k for k in results.keys() if "best_alpha" in results[k]])

# Aggregations
mean_alpha = np.mean(alpha_list)
median_alpha = np.median(alpha_list)
weighted_mean_alpha = np.average(alpha_list, weights=weights)

print("=== Aggregated OOD-informed alphas ===")
print(f"Mean alpha:           {mean_alpha:.2f}")
print(f"Median alpha:         {median_alpha:.2f}")
print(f"Weighted mean alpha:  {weighted_mean_alpha:.2f}")

# Inspect distribution
print("\nPer-subgroup alphas:")
print(alpha_series)


=== Aggregated OOD-informed alphas ===
Mean alpha:           10.08
Median alpha:         5.00
Weighted mean alpha:  9.05

Per-subgroup alphas:
age<45           5.0
male            10.0
female          20.0
high_BP          0.5
high_chol        5.0
exang=1          5.0
cp_2             0.5
cp_3            30.0
cp_4            12.0
low_thalach     10.0
high_thalach     2.0
oldpeak>=2       1.0
restecg_1.0     30.0
dtype: float64


In [54]:
# %%
# --- Cross-validate to select alpha, then fit final models (STOP here) ---
X_np, y_np = X.values, y.values
alphas = [0.01, 0.1, 0.5, 1, 2, 5, 6.5, 6.08, 10.08, 20]

grid_results, summary, best_alpha = cross_val_grid_alphas(
    X_np, y_np, features, targets, alphas, n_splits=5
)

# Fit endelige modeller på hele Hungarian-settet
baseline = LogisticRegression(max_iter=1000).fit(X_np, y_np)
informed = TargetInformedModel(alpha=best_alpha, model_type="logistic").fit(
    X_np, y_np, feature_names=features, targets=targets
)

print("\nModels fitted. Baseline and Target-Informed are ready.")


/Users/solvelangseth/miniconda3/envs/tir-env/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/Users/solvelangseth/miniconda3/envs/tir-env/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/


CV summary (means):
        baseline_accuracy  baseline_log_loss  baseline_auc  informed_accuracy  \
alpha                                                                          
0.01            0.819956           0.408044      0.887856           0.812192   
0.10            0.819956           0.408044      0.887856           0.816110   
0.50            0.819956           0.408044      0.887856           0.819956   
1.00            0.819956           0.408044      0.887856           0.823803   
2.00            0.819956           0.408044      0.887856           0.816183   
5.00            0.819956           0.408044      0.887856           0.823948   
6.08            0.819956           0.408044      0.887856           0.823948   
6.50            0.819956           0.408044      0.887856           0.831640   
10.08           0.819956           0.408044      0.887856           0.831567   
20.00           0.819956           0.408044      0.887856           0.835414   

       informed_l

/Users/solvelangseth/miniconda3/envs/tir-env/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [36]:
cv_results = cross_val_evaluate(X.values, y.values, features, targets, alpha=1.0, n_splits=5)
print("Cross-validation (Hungarian):")
print(cv_results)
print("\nAverage:", cv_results.mean())


/Users/solvelangseth/miniconda3/envs/tir-env/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/Users/solvelangseth/miniconda3/envs/tir-env/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/

Cross-validation (Hungarian):
   baseline_accuracy  baseline_log_loss  baseline_auc  informed_accuracy  \
0           0.811321           0.407987      0.890909           0.811321   
1           0.826923           0.389403      0.889952           0.846154   
2           0.750000           0.572422      0.799043           0.750000   
3           0.826923           0.358578      0.909375           0.826923   
4           0.884615           0.311828      0.950000           0.884615   

   informed_log_loss  informed_auc  
0           0.406920      0.892424  
1           0.403346      0.883573  
2           0.574316      0.807018  
3           0.344812      0.917187  
4           0.305508      0.950000  

Average: baseline_accuracy    0.819956
baseline_log_loss    0.408044
baseline_auc         0.887856
informed_accuracy    0.823803
informed_log_loss    0.406980
informed_auc         0.890040
dtype: float64


/Users/solvelangseth/miniconda3/envs/tir-env/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [38]:
# %%
from sklearn.metrics import accuracy_score, log_loss, roc_auc_score

# Hungarian (train set)
X_hu, y_hu = X.values, y.values

# Cleveland (test set)
X_cl, y_cl = X_cleveland.values, y_cleveland.values

# Fit baseline logistic regression
baseline = LogisticRegression(max_iter=1000).fit(X_hu, y_hu)

# Fit informed logistic regression (α = 10)
informed = TargetInformedModel(alpha=10.0, model_type="logistic").fit(
    X_hu, y_hu, feature_names=features, targets=targets
)

# Predictions
baseline_proba = baseline.predict_proba(X_cl)[:, 1]
informed_proba = informed.predict(X_cl)

# Evaluation
print("=== Cleveland Generalization (Hungarian-trained) ===")
print(f"Baseline  -> Acc: {accuracy_score(y_cl, baseline_proba > 0.5):.3f}  "
      f"LogLoss: {log_loss(y_cl, baseline_proba):.3f}  "
      f"AUC: {roc_auc_score(y_cl, baseline_proba):.3f}")

print(f"Informed  -> Acc: {accuracy_score(y_cl, informed_proba > 0.5):.3f}  "
      f"LogLoss: {log_loss(y_cl, informed_proba):.3f}  "
      f"AUC: {roc_auc_score(y_cl, informed_proba):.3f}")


=== Cleveland Generalization (Hungarian-trained) ===
Baseline  -> Acc: 0.779  LogLoss: 0.486  AUC: 0.847
Informed  -> Acc: 0.779  LogLoss: 0.455  AUC: 0.864


/Users/solvelangseth/miniconda3/envs/tir-env/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
